# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way. 

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [1]:
import s3fs
import boto3
import pandas as pd
import nltk

import re
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import tensorflow as tf
import transformers
import numpy as np


2022-08-01 17:24:41.304398: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-01 17:24:41.467005: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 17:24:41.467022: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import torch

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [3]:
dataset = pd.read_csv('BERTopic_Labeled.csv')

In [4]:
dataset.rename(columns = {'Unnamed: 0':'ID'}, inplace = True)

In [5]:
dataset = dataset[['ID','description', 'university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship']]

In [6]:
dataset['description']

0      From the Wall Street Journal and #1 Amazon bes...
1      Helping set the stage for BioWare's hotly anti...
2      Sebastian Locke, the fifty-six-year-old patria...
3      "Take me back to Oxmoon, the lost paradise of ...
4      When the Mayflower set sail in 1620, it carrie...
                             ...                        
995    Lee wants to be a Tarantula – a member of the ...
996    The Merry Adventures of Robin Hood of Great Re...
997    Moving from present-day Oslo to Brooklyn in th...
998    The captivating sequel to INKHEART, the critic...
999    Santa Claus, my dear old friend, you are a thi...
Name: description, Length: 1000, dtype: object

In [7]:
#dataset['description'] = dataset['description'].apply(lambda x: re.sub(r'[^a-zA-Z ]+', ' ', x))

In [8]:
#dataset['description'] = dataset['description'].apply(lambda x: x.lower())

In [9]:
dataset = dataset.replace(np.nan, False)

In [10]:
dataset

,ID,description,university,relationships,break ups,divorce,weddings,death,family,friendship
0,"39,822",From the Wall Street Journal and #1 Amazon bes...,False,True,False,False,False,False,False,False
1,"34,235",Helping set the stage for BioWare's hotly anti...,False,False,False,False,False,False,False,False
2,"27,904","Sebastian Locke, the fifty-six-year-old patria...",False,True,False,True,False,True,True,False
3,"10,515","""Take me back to Oxmoon, the lost paradise of ...",False,True,False,False,False,False,True,False
4,935,"When the Mayflower set sail in 1620, it carrie...",False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
995,"17,361",Lee wants to be a Tarantula – a member of the ...,False,False,False,False,False,False,False,True
996,"9,029",The Merry Adventures of Robin Hood of Great Re...,False,False,False,False,False,False,False,False
997,"32,216",Moving from present-day Oslo to Brooklyn in th...,False,False,False,False,False,False,True,False
998,"1,036","The captivating sequel to INKHEART, the critic...",False,False,False,False,False,False,False,False


In [11]:
dataset[0:499].to_csv('FINAL_bert_train.csv')
dataset[500:599].to_csv('FINAL_bert_validation.csv')
dataset[600:999].to_csv('FINAL_bert_test.csv')

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [12]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'FINAL_bert_train.csv', 'validation':'FINAL_bert_validation.csv'})

Using custom data configuration default-e7c5fbafc91872b6


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/csv/default-e7c5fbafc91872b6/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Let's check the first example of the training split:

In [13]:
example = dataset['train'][0]
example

{'Unnamed: 0': 0,
 'ID': '39,822',
 'description': 'From the Wall Street Journal and #1 Amazon bestselling author comes a new installment to the Blue Moon small town romance series.  A womanizing bad boy with a motorcycle and sexy-as-sin smile is not part of Emma’s life plan.  She moved cross-country to be close to family and finally settle down in hippie, trippy, nosey Blue Moon Bend. But when famed fashion photographer Niko shows up with his leather jacket, underwear-melting voice, and a problem, she sees nothing but trouble.  He’s all wrong for Emma, but that doesn’t stop the attraction from boiling over.  Niko doesn’t let being friend zoned get in his way. Once he gets his hands—and his mouth—on her, will their friendship survive? Or will he lose everything he’s worked for back in New York to the redhead who dominates his every thought? ',
 'university': False,
 'relationships': True,
 'break ups': False,
 'divorce': False,
 'weddings': False,
 'death': False,
 'family': False,
 'f

The dataset consists of tweets, labeled with one or more emotions. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [14]:
labels = [label for label in dataset['train'][0].keys() if label not in ['ID', 'description', 'Unnamed: 0']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['university',
 'relationships',
 'break ups',
 'divorce',
 'weddings',
 'death',
 'family',
 'friendship']

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'ID', 'description', 'university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'],
        num_rows: 499
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'ID', 'description', 'university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'],
        num_rows: 99
    })
})

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [16]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples['description']
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [17]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Parameter 'function'=<function preprocess_data at 0x7f1228139c10> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [19]:
tokenizer.decode(example['input_ids'])

'[CLS] from the wall street journal and # 1 amazon bestselling author comes a new installment to the blue moon small town romance series. a womanizing bad boy with a motorcycle and sexy - as - sin smile is not part of emma ’ s life plan. she moved cross - country to be close to family and finally settle down in hippie, trippy, nosey blue moon bend. but when famed fashion photographer niko shows up with his leather jacket, underwear - melting voice, and a problem, she sees nothing but trouble. he ’ s all wrong for emma, but that doesn ’ t stop the attraction from boiling over [SEP]'

In [20]:
example['labels']

[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [21]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['relationships']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [22]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [23]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [24]:
batch_size = 8
metric_name = "f1"

In [25]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [26]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [27]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [28]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  2013,  1996,  2813,  2395,  3485,  1998,  1001,  1015,  9733,
         2190, 23836,  2075,  3166,  3310,  1037,  2047, 18932,  2000,  1996,
         2630,  4231,  2235,  2237,  7472,  2186,  1012,  1037,  2450,  6026,
         2919,  2879,  2007,  1037,  9055,  1998,  7916,  1011,  2004,  1011,
         8254,  2868,  2003,  2025,  2112,  1997,  5616,  1521,  1055,  2166,
         2933,  1012,  2016,  2333,  2892,  1011,  2406,  2000,  2022,  2485,
         2000,  2155,  1998,  2633,  7392,  2091,  1999,  5099, 14756,  1010,
         4440,  7685,  1010,  4451,  2100,  2630,  4231,  8815,  1012,  2021,
         2043, 15607,  4827,  8088, 23205,  2080,  3065,  2039,  2007,  2010,
         5898,  6598,  1010, 14236,  1011, 13721,  2376,  1010,  1998,  1037,
         3291,  1010,  2016,  5927,  2498,  2021,  4390,  1012,  2002,  1521,
         1055,  2035,  3308,  2005,  5616,  1010,  2021,  2008,  2987,  1521,
         1056,  2644,  1996,  8432,  2013, 16018,  2058,   102])

In [29]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7021, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.0293, -0.3861,  0.1062, -0.2864,  0.4190, -0.2009, -0.4986,  0.0125]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [30]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [31]:
trainer.train()

/home/ec2-user/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 499
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 315


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.280225,0.000000,0.500000,0.393939
2,No log,0.244740,0.208333,0.558159,0.434343
3,No log,0.230291,0.409836,0.641876,0.454545
4,No log,0.221997,0.448000,0.660168,0.474747
5,No log,0.216404,0.393162,0.631793,0.464646


***** Running Evaluation *****
  Num examples = 99
  Batch size = 8
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-63
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-63/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-63/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-63/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-63/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 99
  Batch size = 8
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-126
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-126/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-126/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-126/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-126/speci

TrainOutput(global_step=315, training_loss=0.28601502918061755, metrics={'train_runtime': 1019.1994, 'train_samples_per_second': 2.448, 'train_steps_per_second': 0.309, 'total_flos': 164124361943040.0, 'train_loss': 0.28601502918061755, 'epoch': 5.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [32]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 99
  Batch size = 8


{'eval_loss': 0.22199691832065582,
 'eval_f1': 0.448,
 'eval_roc_auc': 0.6601683270353831,
 'eval_accuracy': 0.47474747474747475,
 'eval_runtime': 10.1031,
 'eval_samples_per_second': 9.799,
 'eval_steps_per_second': 1.287,
 'epoch': 5.0}

## Inference

Let's test the model on a new sentence:

In [67]:
test_data = pd.read_csv('BERTopic_Labeled.csv')[600:1000].drop(columns = ['university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'])

In [68]:
test_data

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,index,Unnamed: 0,bookId,title,series,author,rating,...,Dystopia,Poetry,Biography,Manga,Thriller,Graphic Novels,Romance,labeled?,Contains True?,tokens
600,600,600,600,600,"39,326",23908423-onyx-webb,Onyx Webb: Episode One: The Story Begins,Onyx Webb,"Richard Fenton (Goodreads Author), Andrea Walt...",4.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"['bestselling', 'authors', 'fenton', 'waltz', ..."
601,601,601,601,601,"34,185",18274597-the-war-inside,The War Inside,The Horizons Trilogy #1,M. Kircher (Goodreads Author),3.92,...,Dystopia,NaN,NaN,NaN,NaN,NaN,Romance,NaN,1.0,"['end', 'world', 'beginning', 'eighteenyearold..."
602,602,602,602,602,"42,881",6294553-deep-kiss-of-winter,Deep Kiss of Winter,Immortals After Dark #7,"Kresley Cole (Goodreads Author), Gena Showalte...",4.22,...,NaN,NaN,NaN,NaN,NaN,NaN,Romance,NaN,1.0,"['kresley', 'cole', 'untouchable', 'immortals'..."
603,603,603,603,603,"25,586",37569307-the-evolution-of-claire,The Evolution of Claire,Jurassic World: Fallen Kingdom #1,Tess Sharpe (Goodreads Author),4.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"['discover', 'beginnings', 'one', 'jurassic', ..."
604,604,604,604,604,"4,579",47954.The_Invisible_Ring,The Invisible Ring,The Black Jewels #4,Anne Bishop,4.08,...,NaN,NaN,NaN,NaN,NaN,NaN,Romance,NaN,1.0,"['prequel', 'set', 'dark', 'sensual', 'world',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,995,995,995,"17,361",588326.The_Blue_Helmet,The Blue Helmet,NaN,William Bell,3.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"['lee', 'wants', 'tarantula', 'member', 'bigge..."
996,996,996,996,996,"9,029",93007.The_Merry_Adventures_of_Robin_Hood,The Merry Adventures of Robin Hood,NaN,Howard Pyle,4.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"['merry', 'adventures', 'robin', 'hood', 'grea..."
997,997,997,997,997,"32,216",1085376.Before_You_Sleep,Before You Sleep,NaN,"Linn Ullmann, Tiina Nunnally (Translator)",3.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"['moving', 'presentday', 'oslo', 'brooklyn', '..."
998,998,998,998,998,"1,036",28195.Inkspell,Inkspell,Inkworld #2,"Cornelia Funke (Goodreads Author), Anthea Bell...",3.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"['captivating', 'sequel', 'inkheart', 'critica..."


In [69]:
def predict_bert(text):
    #text = final_preds['description'][i]
    encoding = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
    outputs = trainer.model(**encoding)
    logits = outputs.logits
    #print(i, logits.shape)
    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    return probs.tolist()

In [71]:
test_data['scores'] = test_data['description'].apply(lambda x: predict_bert(x))

In [72]:
test_data

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,index,Unnamed: 0,bookId,title,series,author,rating,...,Poetry,Biography,Manga,Thriller,Graphic Novels,Romance,labeled?,Contains True?,tokens,scores
600,600,600,600,600,"39,326",23908423-onyx-webb,Onyx Webb: Episode One: The Story Begins,Onyx Webb,"Richard Fenton (Goodreads Author), Andrea Walt...",4.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"['bestselling', 'authors', 'fenton', 'waltz', ...","[0.04233139380812645, 0.39226487278938293, 0.0..."
601,601,601,601,601,"34,185",18274597-the-war-inside,The War Inside,The Horizons Trilogy #1,M. Kircher (Goodreads Author),3.92,...,NaN,NaN,NaN,NaN,NaN,Romance,NaN,1.0,"['end', 'world', 'beginning', 'eighteenyearold...","[0.040408384054899216, 0.408073753118515, 0.02..."
602,602,602,602,602,"42,881",6294553-deep-kiss-of-winter,Deep Kiss of Winter,Immortals After Dark #7,"Kresley Cole (Goodreads Author), Gena Showalte...",4.22,...,NaN,NaN,NaN,NaN,NaN,Romance,NaN,1.0,"['kresley', 'cole', 'untouchable', 'immortals'...","[0.06058784946799278, 0.6148503422737122, 0.03..."
603,603,603,603,603,"25,586",37569307-the-evolution-of-claire,The Evolution of Claire,Jurassic World: Fallen Kingdom #1,Tess Sharpe (Goodreads Author),4.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"['discover', 'beginnings', 'one', 'jurassic', ...","[0.04561688378453255, 0.3967863619327545, 0.02..."
604,604,604,604,604,"4,579",47954.The_Invisible_Ring,The Invisible Ring,The Black Jewels #4,Anne Bishop,4.08,...,NaN,NaN,NaN,NaN,NaN,Romance,NaN,1.0,"['prequel', 'set', 'dark', 'sensual', 'world',...","[0.0368616096675396, 0.24831132590770721, 0.02..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,995,995,995,"17,361",588326.The_Blue_Helmet,The Blue Helmet,NaN,William Bell,3.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"['lee', 'wants', 'tarantula', 'member', 'bigge...","[0.06012316420674324, 0.475518137216568, 0.026..."
996,996,996,996,996,"9,029",93007.The_Merry_Adventures_of_Robin_Hood,The Merry Adventures of Robin Hood,NaN,Howard Pyle,4.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"['merry', 'adventures', 'robin', 'hood', 'grea...","[0.045951638370752335, 0.14277443289756775, 0...."
997,997,997,997,997,"32,216",1085376.Before_You_Sleep,Before You Sleep,NaN,"Linn Ullmann, Tiina Nunnally (Translator)",3.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"['moving', 'presentday', 'oslo', 'brooklyn', '...","[0.08235049992799759, 0.6309429407119751, 0.04..."
998,998,998,998,998,"1,036",28195.Inkspell,Inkspell,Inkworld #2,"Cornelia Funke (Goodreads Author), Anthea Bell...",3.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"['captivating', 'sequel', 'inkheart', 'critica...","[0.03647300601005554, 0.24650506675243378, 0.0..."


In [73]:
bert_list = ['university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship']
#final_preds['scores'] = test_probabilities
test_data['labels'] = str(bert_list)

In [74]:
from ast import literal_eval
test_data['labels'] = test_data['labels'].apply(lambda row: literal_eval(row))
#test_data['scores'] = test_data['scores'].apply(lambda row: literal_eval(row))
test_data['dictionary'] = test_data.apply(lambda row: dict(zip(row['labels'], row['scores'])), axis=1)
LE_columns = test_data['dictionary'].apply(pd.Series)
LE_columns = LE_columns > 0.5
fine_tuned_8_1 = pd.concat([test_data, LE_columns], axis=1)

In [75]:
fine_tuned_8_1

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,index,Unnamed: 0,bookId,title,series,author,rating,...,labels,dictionary,university,relationships,break ups,divorce,weddings,death,family,friendship
600,600,600,600,600,"39,326",23908423-onyx-webb,Onyx Webb: Episode One: The Story Begins,Onyx Webb,"Richard Fenton (Goodreads Author), Andrea Walt...",4.00,...,"[university, relationships, break ups, divorce...","{'university': 0.04233139380812645, 'relations...",False,False,False,False,False,False,False,False
601,601,601,601,601,"34,185",18274597-the-war-inside,The War Inside,The Horizons Trilogy #1,M. Kircher (Goodreads Author),3.92,...,"[university, relationships, break ups, divorce...","{'university': 0.040408384054899216, 'relation...",False,False,False,False,False,False,False,False
602,602,602,602,602,"42,881",6294553-deep-kiss-of-winter,Deep Kiss of Winter,Immortals After Dark #7,"Kresley Cole (Goodreads Author), Gena Showalte...",4.22,...,"[university, relationships, break ups, divorce...","{'university': 0.06058784946799278, 'relations...",False,True,False,False,False,False,False,False
603,603,603,603,603,"25,586",37569307-the-evolution-of-claire,The Evolution of Claire,Jurassic World: Fallen Kingdom #1,Tess Sharpe (Goodreads Author),4.05,...,"[university, relationships, break ups, divorce...","{'university': 0.04561688378453255, 'relations...",False,False,False,False,False,False,False,False
604,604,604,604,604,"4,579",47954.The_Invisible_Ring,The Invisible Ring,The Black Jewels #4,Anne Bishop,4.08,...,"[university, relationships, break ups, divorce...","{'university': 0.0368616096675396, 'relationsh...",False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,995,995,995,"17,361",588326.The_Blue_Helmet,The Blue Helmet,NaN,William Bell,3.42,...,"[university, relationships, break ups, divorce...","{'university': 0.06012316420674324, 'relations...",False,False,False,False,False,False,False,False
996,996,996,996,996,"9,029",93007.The_Merry_Adventures_of_Robin_Hood,The Merry Adventures of Robin Hood,NaN,Howard Pyle,4.07,...,"[university, relationships, break ups, divorce...","{'university': 0.045951638370752335, 'relation...",False,False,False,False,False,False,False,False
997,997,997,997,997,"32,216",1085376.Before_You_Sleep,Before You Sleep,NaN,"Linn Ullmann, Tiina Nunnally (Translator)",3.34,...,"[university, relationships, break ups, divorce...","{'university': 0.08235049992799759, 'relations...",False,True,False,False,False,False,False,False
998,998,998,998,998,"1,036",28195.Inkspell,Inkspell,Inkworld #2,"Cornelia Funke (Goodreads Author), Anthea Bell...",3.91,...,"[university, relationships, break ups, divorce...","{'university': 0.03647300601005554, 'relations...",False,False,False,False,False,False,False,False


In [76]:
fine_tuned_8_1.to_csv('fine_tuned_8_1.csv')